In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [ ]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16429 sha256=1e90a68d05bfab42af389cdc957a6a0f388011a850442b86a3049e9ae14e489d
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet
import torch.nn as nn

# 모델 초기화
num_classes = 3
model = EfficientNet.from_pretrained('efficientnet-b5')

# EfficientNet 모델의 분류기 변경
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, num_classes)

# 드롭아웃 레이어 추가
dropout = nn.Dropout(0.5)  # 드롭아웃 레이어 생성
model._dropout = dropout  # 모델에 추가

# 파라미터 수 계산
pretrained_params = sum(p.numel() for p in model.parameters())
num_classes = 3
dropout_params = sum(p.numel() for p in dropout.parameters())
total_params = pretrained_params + num_ftrs * num_classes + num_classes + dropout_params

print("사전 훈련된 모델의 파라미터 수:", pretrained_params)
print("새로운 출력 레이어의 입력 피쳐의 크기:", num_ftrs)
print("분류할 클래스의 개수:", num_classes)
print("드롭아웃 레이어의 파라미터 수:", dropout_params)
print("최종 모델의 파라미터 수:", total_params)


Loaded pretrained weights for efficientnet-b5
사전 훈련된 모델의 파라미터 수: 28346931
새로운 출력 레이어의 입력 피쳐의 크기: 2048
분류할 클래스의 개수: 3
드롭아웃 레이어의 파라미터 수: 0
최종 모델의 파라미터 수: 28353078


# resnet50
- 모델의 Fully Connected Layer를 변경하여 출력 레이어를 새로운 분류를 수행할 수 있도록 합니다.
- 드롭아웃 레이어를 추가합니다.

- 사전 훈련된 모델의 파라미터 수 :
- 새로운 분류를 위한 출력 레이어의 파라미터 수 :
- 추가된 드롭아웃 레이어의 파라미터 수 :

In [ ]:
import torch.nn as nn
from torchvision import models

# 모델 초기화
num_classes = 3
model = models.resnet50(pretrained=True)

# ResNet 모델의 Fully Connected Layer 변경
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# 드롭아웃 레이어 추가
dropout = nn.Dropout(0.5)  # 드롭아웃 레이어 생성
model.dropout = dropout  # 모델에 추가

# 파라미터 수 계산
pretrained_params = sum(p.numel() for p in model.parameters())
num_classes = 3
dropout_params = sum(p.numel() for p in dropout.parameters())
total_params = pretrained_params + num_ftrs * num_classes + num_classes + dropout_params

print("사전 훈련된 모델의 파라미터 수:", pretrained_params)
print("새로운 출력 레이어의 입력 피쳐의 크기:", num_ftrs)
print("분류할 클래스의 개수:", num_classes)
print("드롭아웃 레이어의 파라미터 수:", dropout_params)
print("최종 모델의 파라미터 수:", total_params)


사전 훈련된 모델의 파라미터 수: 23514179
새로운 출력 레이어의 입력 피쳐의 크기: 2048
분류할 클래스의 개수: 3
드롭아웃 레이어의 파라미터 수: 0
최종 모델의 파라미터 수: 23520326


In [ ]:
import torch
import torch.nn as nn
import timm

# ViT 모델 불러오기
model = timm.create_model('vit_base_patch16_224', pretrained=True)

# 드롭아웃 확률 설정
dropout_prob = 0.5

# 기존 분류 레이어의 입력 특성 수 가져오기
in_features = model.head.in_features

# 새로운 분류 레이어 정의
num_classes = 3  # 새로운 클래스 수
classifier = nn.Sequential(
    nn.Dropout(dropout_prob),
    nn.Linear(in_features, num_classes)
)

# 모델의 분류 레이어를 새로운 분류기로 교체
model.head = classifier

# 파라미터 수 계산
pretrained_params = sum(p.numel() for p in model.parameters())
dropout_params = sum(p.numel() for p in classifier[0].parameters())  # 드롭아웃 레이어의 파라미터 수
num_classes = 3  # 새로운 클래스 수

total_params = pretrained_params + in_features * num_classes + num_classes + dropout_params

print("사전 훈련된 모델의 파라미터 수:", pretrained_params)
print("새로운 출력 레이어의 입력 피쳐의 크기:", in_features)
print("분류할 클래스의 개수:", num_classes)
print("드롭아웃 레이어의 파라미터 수:", dropout_params)
print("최종 모델의 파라미터 수:", total_params)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

사전 훈련된 모델의 파라미터 수: 85800963
새로운 출력 레이어의 입력 피쳐의 크기: 768
분류할 클래스의 개수: 3
드롭아웃 레이어의 파라미터 수: 0
최종 모델의 파라미터 수: 85803270
